## Cleaning and Loading of the CSV Files into a Table in a Database

A company (a clone of deliveroo), delivers takeaway from restaurants, has all their data in separate csv files. The data describes customers, the  restaurants delivering take out, and the orders. They wish to convert the csv files into a relational database. The data files have been downloaded from the link provided.

A Python script will be written to;
* read the CSV files (thoroughly review to understand the structure and content of the files)
* clean up the datasets by removing the referred columns

In [2]:
import pandas as pd

# Function to load and inspect a dataset
def load_and_inspect(file_path, nrows=None):
    # Load dataset
    data = pd.read_csv(file_path, nrows=nrows, low_memory=False)
    # Display structure
    print(f"Dataset: {file_path}")
    print(f"Shape: {data.shape}")
    print("Columns:")
    print(data.columns)
    print("\nSample data:")
    print(data.head())
    print("\n")
    return data

# Load and inspect datasets
customers = load_and_inspect("train_customers.csv")
orders = load_and_inspect("orders.csv")
vendors = load_and_inspect("vendors.csv")
locations = load_and_inspect("train_locations.csv")

# Load train_full.csv in chunks to manage memory usage
full_chunks = pd.read_csv("train_full.csv", chunksize=10000, low_memory=False)
full = pd.concat(full_chunks, ignore_index=True)

# Inspect train_full after combining chunks
print("Dataset: train_full.csv")
print(f"Shape: {full.shape}")
print("Columns:")
print(full.columns)
print("\nSample data:")
print(full.head())



Dataset: train_customers.csv
Shape: (34674, 8)
Columns:
Index(['akeed_customer_id', 'gender', 'dob', 'status', 'verified', 'language',
       'created_at', 'updated_at'],
      dtype='object')

Sample data:
  akeed_customer_id gender  dob  status  verified language  \
0           TCHWPBT   Male  NaN       1         1       EN   
1           ZGFSYCZ   Male  NaN       1         1       EN   
2           S2ALZFL   Male  NaN       0         1       EN   
3           952DBJQ   Male  NaN       1         1       EN   
4           1IX6FXS   Male  NaN       1         1       EN   

            created_at           updated_at  
0  2018-02-07 19:16:23  2018-02-07 19:16:23  
1  2018-02-09 12:04:42  2018-02-09 12:04:41  
2  2018-03-14 18:31:43  2018-03-14 18:31:42  
3  2018-03-15 19:47:07  2018-03-15 19:47:07  
4  2018-03-15 19:57:01  2018-03-15 19:57:01  


Dataset: orders.csv
Shape: (135303, 26)
Columns:
Index(['akeed_order_id', 'customer_id', 'item_count', 'grand_total',
       'payment_mode', '

In [4]:
# Specify columns to drop based on the project requirements
columns_to_drop = [
    'sunday_from_time1', 'sunday_to_time1', 'sunday_from_time2', 'sunday_to_time2',
    'monday_from_time1', 'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
    'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2', 'tuesday_to_time2',
    'wednesday_from_time1', 'wednesday_to_time1', 'wednesday_from_time2', 'wednesday_to_time2',
    'thursday_from_time1', 'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
    'friday_from_time1', 'friday_to_time1', 'friday_from_time2', 'friday_to_time2',
    'saturday_from_time1', 'saturday_to_time1', 'saturday_from_time2', 'saturday_to_time2'
]

# Drop unwanted columns
vendors = vendors.drop(columns=[col for col in columns_to_drop if col in vendors.columns], errors='ignore')
full = full.drop(columns=[col for col in columns_to_drop if col in full.columns], errors='ignore')
# errors='ignore' is used to handle cases where a column does not exist in a dataset, avoiding errors

# Display cleaned datasets structure
print("\nCleaned Vendors Columns:")
print(vendors.columns)
print("\nCleaned Train_Full Columns:")
print(full.columns)




Cleaned Vendors Columns:
Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'primary_tags', 'open_close_flags', 'vendor_tag',
       'vendor_tag_name', 'one_click_vendor', 'country_id', 'city_id',
       'created_at', 'updated_at', 'device_type', 'display_orders'],
      dtype='object')

Cleaned Train_Full Columns:
Index(['customer_id', 'gender', 'status_x', 'verified_x', 'created_at_x',
       'updated_at_x', 'location_number', 'location_type', 'latitude_x',
       'longitude_x', 'id', 'authentication_id', 'latitude_y', 'longitude_y',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prep

In [6]:
# Checking for missing values across the files
import pandas as pd

# List of datasets for inspection
datasets = {
    "customers": customers,
    "orders": orders,
    "vendors": vendors,
    "locations": locations,
    "full": full
}

# Check for missing values
for name, dataset in datasets.items():
    print(f"Missing values in {name}:")
    print(dataset.isnull().sum())
    print("-" * 50)


Missing values in customers:
akeed_customer_id        0
gender               12154
dob                  31628
status                   0
verified                 0
language             13575
created_at               0
updated_at               0
dtype: int64
--------------------------------------------------
Missing values in orders:
akeed_order_id                        70
customer_id                            0
item_count                          6925
grand_total                            0
payment_mode                           0
promo_code                        130998
vendor_discount_amount                 0
promo_code_discount_percentage     69423
is_favorite                        35195
is_rated                               0
vendor_rating                      90083
driver_rating                          0
deliverydistance                       0
preparationtime                    55560
delivery_time                     130180
order_accepted_time                48348
driver_ac

In [8]:
# Handle Missing Data in critical columns
import pandas as pd

# Define critical columns for each dataset
critical_columns = {
    "customers": ["akeed_customer_id"],  # Primary key
    "orders": ["akeed_order_id", "customer_id", "vendor_id", "LOCATION_NUMBER", "LOCATION_TYPE"],  # Primary/foreign keys
    "vendors": ["id"],  # Primary key
    "locations": ["customer_id", "location_number", "location_type"],  # Foreign key/composite primary keys
    "full": ["customer_id", "location_number", "location_type", "id"],  # Keys and important references
}

# Drop rows with missing values in critical columns

for name, dataset in datasets.items():
    print(f"Before cleaning: {name} - {dataset.shape}")
    critical_cols = critical_columns[name]
    datasets[name] = dataset.dropna(subset=critical_cols)
    print(f"After cleaning: {name} - {datasets[name].shape}")
    print("-" * 50)

# Updated datasets: customers, orders, vendors, locations, full
customers = datasets["customers"]
orders = datasets["orders"]
vendors = datasets["vendors"]
locations = datasets["locations"]
full = datasets["full"]


Before cleaning: customers - (34674, 8)
After cleaning: customers - (34674, 8)
--------------------------------------------------
Before cleaning: orders - (135303, 26)
After cleaning: orders - (86364, 26)
--------------------------------------------------
Before cleaning: vendors - (100, 31)
After cleaning: vendors - (100, 31)
--------------------------------------------------
Before cleaning: locations - (59503, 5)
After cleaning: locations - (32294, 5)
--------------------------------------------------
Before cleaning: full - (5802400, 45)
After cleaning: full - (3148200, 45)
--------------------------------------------------


In [10]:
import pandas as pd

# Check data types for each dataset
print("Customers Data Types:")
print(customers.dtypes)
print("\nOrders Data Types:")
print(orders.dtypes)
print("\nVendors Data Types:")
print(vendors.dtypes)
print("\nLocations Data Types:")
print(locations.dtypes)
print("\nFull Data Types:")
print(full.dtypes)

# Ensure consistency in key columns
# Check key columns' data types
key_columns = {
    "customer_id": [customers["akeed_customer_id"].dtype, full["customer_id"].dtype, orders["customer_id"].dtype, locations["customer_id"].dtype],
    "location_number": [locations["location_number"].dtype, full["location_number"].dtype, orders["LOCATION_NUMBER"].dtype],
    "location_type": [locations["location_type"].dtype, full["location_type"].dtype, orders["LOCATION_TYPE"].dtype],
    "vendor_id": [vendors["id"].dtype, full["id"].dtype]  
}

# Print data types for key columns
for key, dtypes in key_columns.items():
    print(f"Key Column: {key}")
    print(f"Data Types: {dtypes}\n")


Customers Data Types:
akeed_customer_id     object
gender                object
dob                  float64
status                 int64
verified               int64
language              object
created_at            object
updated_at            object
dtype: object

Orders Data Types:
akeed_order_id                    float64
customer_id                        object
item_count                        float64
grand_total                       float64
payment_mode                        int64
promo_code                         object
vendor_discount_amount            float64
promo_code_discount_percentage    float64
is_favorite                        object
is_rated                           object
vendor_rating                     float64
driver_rating                     float64
deliverydistance                  float64
preparationtime                   float64
delivery_time                      object
order_accepted_time                object
driver_accepted_time               objec

In [12]:
# List of datasets and their names
datasets = [customers, orders, vendors, locations, full]
dataset_names = ["customers", "orders", "vendors", "locations", "full"]

# Loop through each dataset to identify numerical and categorical columns
for i, dataset in enumerate(datasets):
    print(f"\nDataset: {dataset_names[i]}")
    
    # Identify numerical columns
    numerical_columns = dataset.select_dtypes(include=['int64', 'float64']).columns.tolist()
    print("Numerical Columns:")
    print(numerical_columns)
    
    # Identify categorical columns
    categorical_columns = dataset.select_dtypes(include=['object']).columns.tolist()
    print("Categorical Columns:")
    print(categorical_columns)




Dataset: customers
Numerical Columns:
['dob', 'status', 'verified']
Categorical Columns:
['akeed_customer_id', 'gender', 'language', 'created_at', 'updated_at']

Dataset: orders
Numerical Columns:
['akeed_order_id', 'item_count', 'grand_total', 'payment_mode', 'vendor_discount_amount', 'promo_code_discount_percentage', 'vendor_rating', 'driver_rating', 'deliverydistance', 'preparationtime', 'vendor_id', 'LOCATION_NUMBER']
Categorical Columns:
['customer_id', 'promo_code', 'is_favorite', 'is_rated', 'delivery_time', 'order_accepted_time', 'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time', 'delivered_time', 'delivery_date', 'created_at', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR']

Dataset: vendors
Numerical Columns:
['id', 'authentication_id', 'latitude', 'longitude', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'prepration_time', 'commission', 'discount_percentage', 'status', 'verified', 'rank', 'vendor_rating', 'open_close_flags', 'country_id

In [14]:
# Handling Missing Values in the Non-critical columns
import pandas as pd

# Threshold for dropping columns with missing values
threshold = 0.5

# Loop through each dataset for processing
for i, dataset in enumerate(datasets):
    print(f"\nProcessing dataset: {dataset_names[i]}")

    # Calculate missing value ratio for each column
    missing_ratio = dataset.isnull().mean()

    # Identify columns to drop, except 'dob'
    to_drop = missing_ratio[missing_ratio > threshold].index.difference(['dob'])

    # Drop identified columns
    dataset = dataset.drop(columns=to_drop)
    print(f"Columns dropped in {dataset_names[i]}: {list(to_drop)}")

    # Handle missing values
    for col in dataset.columns:
        if col == 'dob':  # Special handling for 'dob'
            dataset[col] = dataset[col].fillna(0).astype(int)  # Fill with 0 and convert to integer
        elif dataset[col].dtype in ['int64', 'float64']:  # Numeric columns
            dataset[col] = dataset[col].fillna(dataset[col].mean())  # Fill with mean
        elif dataset[col].dtype == 'object':  # Categorical columns
            dataset[col] = dataset[col].fillna("Not Specified")  # Fill with "Not Specified"

    # Update the dataset in the list
    datasets[i] = dataset

    # Print the updated dataset for verification
    print(f"Updated {dataset_names[i]} dataset:")
    print(datasets[i].head())



Processing dataset: customers
Columns dropped in customers: []
Updated customers dataset:
  akeed_customer_id gender  dob  status  verified language  \
0           TCHWPBT   Male    0       1         1       EN   
1           ZGFSYCZ   Male    0       1         1       EN   
2           S2ALZFL   Male    0       0         1       EN   
3           952DBJQ   Male    0       1         1       EN   
4           1IX6FXS   Male    0       1         1       EN   

            created_at           updated_at  
0  2018-02-07 19:16:23  2018-02-07 19:16:23  
1  2018-02-09 12:04:42  2018-02-09 12:04:41  
2  2018-03-14 18:31:43  2018-03-14 18:31:42  
3  2018-03-15 19:47:07  2018-03-15 19:47:07  
4  2018-03-15 19:57:01  2018-03-15 19:57:01  

Processing dataset: orders
Columns dropped in orders: ['delivery_date', 'delivery_time', 'driver_accepted_time', 'promo_code', 'promo_code_discount_percentage', 'vendor_rating']
Updated orders dataset:
   akeed_order_id customer_id  item_count  grand_total  p

In [16]:
# Handling date columns
import pandas as pd

# Convert date columns to datetime format
date_columns = {
    "customers": ["created_at", "updated_at"],
    "orders": [
        "order_accepted_time","ready_for_pickup_time", "picked_up_time", "delivered_time",
        "created_at"
    ],
    "vendors": ["created_at", "updated_at", "OpeningTime", "OpeningTime2"],
    "locations": [],
    "full": ["created_at_x", "updated_at_x", "created_at_y", "updated_at_y"]
}

for i, dataset in enumerate(datasets):
    for date_col in date_columns[dataset_names[i]]:
        if date_col in dataset.columns:
            datasets[i][date_col] = pd.to_datetime(dataset[date_col], errors='coerce', utc=True)
        else:
            print(f"Column '{date_col}' not found in dataset '{dataset_names[i]}'. Skipping...")

# Check for remaining missing values and data types
for i, dataset in enumerate(datasets):
    print(f"\n{dataset_names[i]} - Missing Values:")
    print(dataset.isnull().sum())
    print(f"\n{dataset_names[i]} - Data Types:")
    print(dataset.dtypes)


C:\Users\USER\AppData\Local\Temp\ipykernel_9164\1842184828.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datasets[i][date_col] = pd.to_datetime(dataset[date_col], errors='coerce', utc=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_9164\1842184828.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datasets[i][date_col] = pd.to_datetime(dataset[date_col], errors='coerce', utc=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_9164\1842184828.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datasets[i][date_col] = pd.to_datetime(dataset[date_col], errors='coerce',


customers - Missing Values:
akeed_customer_id    0
gender               0
dob                  0
status               0
verified             0
language             0
created_at           0
updated_at           0
dtype: int64

customers - Data Types:
akeed_customer_id                 object
gender                            object
dob                                int32
status                             int64
verified                           int64
language                          object
created_at           datetime64[ns, UTC]
updated_at           datetime64[ns, UTC]
dtype: object

orders - Missing Values:
akeed_order_id                0
customer_id                   0
item_count                    0
grand_total                   0
payment_mode                  0
vendor_discount_amount        0
is_favorite                   0
is_rated                      0
driver_rating                 0
deliverydistance              0
preparationtime               0
order_accepted_time       267

In [18]:
# Handling Missing Timestamps
# Replacing missing timestamps with NaT using assignment to avoid chained assignment warning
timestamp_cols_orders = ['order_accepted_time', 'ready_for_pickup_time', 'picked_up_time', 'delivered_time']
orders[timestamp_cols_orders] = orders[timestamp_cols_orders].apply(lambda col: col.fillna(pd.NaT))

# Correct Data Types
# Converting float64 IDs to Int64 
orders['akeed_order_id'] = orders['akeed_order_id'].astype('Int64')

# Vendors: Normalizing OpeningTime and OpeningTime2 (remove time zone info)
# Converting to datetime
vendors['OpeningTime'] = pd.to_datetime(vendors['OpeningTime'], errors='coerce', utc=True)
vendors['OpeningTime2'] = pd.to_datetime(vendors['OpeningTime2'], errors='coerce', utc=True)

# Removing timezone information after ensuring valid datetime conversion
vendors['OpeningTime'] = vendors['OpeningTime'].dt.tz_localize(None)
vendors['OpeningTime2'] = vendors['OpeningTime2'].dt.tz_localize(None)


# Handling any remaining invalid entries
invalid_opening_time = vendors['OpeningTime'].isna().sum()
if invalid_opening_time > 0:
    print(f"Warning: {invalid_opening_time} invalid entries in 'OpeningTime' were set to NaT.")

invalid_opening_time2 = vendors['OpeningTime2'].isna().sum()
if invalid_opening_time2 > 0:
    print(f"Warning: {invalid_opening_time2} invalid entries in 'OpeningTime2' were set to NaT.")

# Replace invalid times with a default (e.g., midnight)
vendors['OpeningTime'] = vendors['OpeningTime'].fillna(pd.Timestamp("00:00:00"))
vendors['OpeningTime2'] = vendors['OpeningTime2'].fillna(pd.Timestamp("00:00:00"))


# Feature Engineering
# Creating binary flags for missing timestamps
for col in timestamp_cols_orders:
    orders[f'{col}_missing'] = orders[col].isna().astype('int64')

# Validate Relationships
# Checking for mismatches between related datasets
missing_customers = set(orders['customer_id']) - set(customers['akeed_customer_id'])
if missing_customers:
    print(f"Missing customers in 'customers' dataset: {missing_customers}")

# Consistency Checks Across IDs
# Ensure IDs in locations match customers and vendors
missing_location_customers = set(locations['customer_id']) - set(customers['akeed_customer_id'])
if missing_location_customers:
    print(f"Missing customer IDs in 'locations': {missing_location_customers}")

# Ensure location_number consistency
if 'location_number' in orders.columns and 'location_number' in locations.columns:
    missing_location_numbers = set(orders['LOCATION_NUMBER']) - set(locations['location_number'])
    if missing_location_numbers:
        print(f"Missing location numbers in 'locations': {missing_location_numbers}")

# Check for remaining missing values and data types
for i, dataset in enumerate(datasets):
    print(f"\n{dataset_names[i]} - Missing Values:")
    print(dataset.isnull().sum())
    print(f"\n{dataset_names[i]} - Data Types:")
    print(dataset.dtypes)

C:\Users\USER\AppData\Local\Temp\ipykernel_9164\548105323.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vendors['OpeningTime'] = pd.to_datetime(vendors['OpeningTime'], errors='coerce', utc=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_9164\548105323.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vendors['OpeningTime2'] = pd.to_datetime(vendors['OpeningTime2'], errors='coerce', utc=True)


Missing customers in 'customers' dataset: {'OM6ZGOW', '4AW59F4', '58IROXQ', 'B6IFRS1', 'WYJVRX8', '3AKIJ6O', '1M6VTT8', 'KH1NXMX', '1S0B1H3', 'QW17A4C', 'GLWWMFB', 'RWTA9I7', 'KOOATQ3', 'LO4MZBT', 'V3T6F0G', 'H9LH52V', 'IFZB6X8', '26PGDYY', 'HRU7NJS', 'QOZ8CIL', 'U8U89GF', 'QCSUE6V', '7ARDGB2', 'ZCLZ0MQ', 'HRBMGRJ', 'M8J1BOM', '8NMUHLB', '0SIOQGR', 'QB35ZMQ', '7FLFDP2', 'C3O6NGA', 'JJQYJSC', 'J40Y5NT', 'Z6QKMCZ', 'TQUO5ZD', 'XGJCRBC', 'Z16NIW5', 'JNFE1NC', 'T08MSTU', 'LVSDBMQ', 'FEWZVLO', 'CSKN8XP', 'KPDZHDQ', '0VZSLGS', 'O7XIXIJ', 'Y0MYAYD', 'F1FB5LI', '4I0UN0D', 'ETB9HI2', 'Y5AJ194', '6EFJRRA', 'SML83HZ', 'VS7Q4WG', 'DVI0TQI', 'OK6N5SP', 'A0RWRDG', 'H2MO4EG', 'ZNZVDNJ', 'UQ4B0WM', '8T0S2TA', 'XOP9ACD', '3TKJ3ZC', 'DSF23FH', 'KMT0UOX', 'QJNA59O', 'MYZST8R', 'QWJWZW7', '1G940Y5', '9EUTGPB', '7IWO76F', 'T0ZQ5OT', '79LGVCO', 'KF99S3C', '92WUF1W', '0B3QNQ5', '1IGS0P3', 'LW3UXK2', 'VGU4284', 'M825VSK', '2AM8ARW', 'M85KUEF', 'MSU1Q1A', 'H8SNDI1', 'AFEN3NK', 'TJ7DCY7', 'VWL6FB3', '3A6EODA', 

In [20]:
# Identify unmatched customer IDs in locations
unmatched_locations = locations[~locations['customer_id'].isin(customers['akeed_customer_id'])]

# Identify unmatched customer IDs in orders
unmatched_orders = orders[~orders['customer_id'].isin(customers['akeed_customer_id'])]

print(f"Unmatched customer IDs in locations: {len(unmatched_locations)}")
print(f"Unmatched customer IDs in orders: {len(unmatched_orders)}")

# Adding placeholder customers
unmatched_customer_ids = set(unmatched_locations['customer_id']).union(unmatched_orders['customer_id'])
for cid in unmatched_customer_ids:
    customers = pd.concat([customers, pd.DataFrame({'customer_id': [cid]})], ignore_index=True)


Unmatched customer IDs in locations: 812
Unmatched customer IDs in orders: 2140




The structure and logical relationships between the datasets are as follows:

1 train_customers.csv: Contains customer details.
* Primary Key: akeed_customer_id

2 orders.csv: Contains order details.
* Primary Key: akeed_order_id
* Foreign Keys:
customer_id (links to customers)
vendor_id (links to vendors)

3 vendors.csv: Contains vendor (restaurant) details.
* Primary Key: id
* Foreign Key: None (directly linked to orders)

4 train_locations.csv: Contains location details for customers.
* Primary Key: None
* Foreign Key: customer_id (links to customers)

5 train_full.csv: Contains a combination of orders and vendors for analysis.
* Primary Key: None (likely a transactional table)
* Foreign Keys:
customer_id (links to customers)
location_number (links to locations)
id (links to vendors)


In [22]:
# Inspect column names and data types
import pandas as pd

for i, dataset in enumerate(datasets):
    print(f"\n{dataset_names[i]} - Column Names and Data Types:")
    print(dataset.dtypes)



customers - Column Names and Data Types:
akeed_customer_id                 object
gender                            object
dob                                int32
status                             int64
verified                           int64
language                          object
created_at           datetime64[ns, UTC]
updated_at           datetime64[ns, UTC]
dtype: object

orders - Column Names and Data Types:
akeed_order_id                        float64
customer_id                            object
item_count                            float64
grand_total                           float64
payment_mode                            int64
vendor_discount_amount                float64
is_favorite                            object
is_rated                               object
driver_rating                         float64
deliverydistance                      float64
preparationtime                       float64
order_accepted_time       datetime64[ns, UTC]
ready_for_pickup_time    

In [34]:
# Database creation
import pandas as pd
import sqlite3

# Mapping of pandas data types to SQL data types
def map_dtype_to_sql(dtype):
    if dtype == "int64" or dtype.name == "Int64":
        return "INTEGER"
    elif dtype == "float64":
        return "REAL"
    elif dtype == "object":
        return "TEXT"
    elif "datetime64" in dtype.name:
        return "DATETIME"
    else:
        return "TEXT"

# Define the primary and foreign key constraints
key_definitions = {
    "customers": {
        "primary_key": "akeed_customer_id",
        "foreign_keys": {}
    },
    "orders": {
        "primary_key": "akeed_order_id",
        "foreign_keys": {
            "customer_id": "customers(akeed_customer_id)",
            "location_number, location_type": "locations(location_number, location_type)",
            "vendor_id": "vendors(id)"
        }
    },
    "locations": {
        "primary_key": "location_number, location_type",  # Composite primary key
        "foreign_keys": {
            "customer_id": "customers(akeed_customer_id)"
        }
    },
    "vendors": {
        "primary_key": "id",
        "foreign_keys": {}
    },
    "full": {
        "primary_key": None,  # No primary key
        "foreign_keys": {
            "customer_id": "customers(akeed_customer_id)",
            "location_number, location_type": "locations(location_number, location_type)",
            "id": "vendors(id)"
        }
    }
}

# Database connection
db_name = "project_deliveroo_clone.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Create tables dynamically
def create_tables(datasets, dataset_names, key_definitions, conn):
    for i, dataset in enumerate(datasets):
        table_name = dataset_names[i].lower()
        print(f"Creating table: {table_name}")
        
        # Get the primary and foreign key definitions
        primary_key = key_definitions[table_name]["primary_key"]
        foreign_keys = key_definitions[table_name]["foreign_keys"]

        # Map columns to SQL types
        column_definitions = []
        for column_name in dataset.columns:
            dtype = dataset[column_name].dtype
            sql_type = map_dtype_to_sql(dtype)
            column_definitions.append(f"{column_name} {sql_type}")
        
        # Add primary key
        if primary_key:
            column_definitions.append(f"PRIMARY KEY ({primary_key})")
        
        # Add foreign keys
        for column, reference in foreign_keys.items():
            if "," in column:  # Composite foreign key
                column_definitions.append(f"FOREIGN KEY ({column}) REFERENCES {reference}")
            else:  # Single foreign key
                column_definitions.append(f"FOREIGN KEY ({column}) REFERENCES {reference}")

        # Create the SQL CREATE TABLE statement
        create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_definitions)}
        );
        """
        try:
            print(create_table_sql)  # Print for debugging
            cursor.execute(create_table_sql)
        except sqlite3.OperationalError as e:
            print(f"Error creating table {table_name}: {e}")
    
    conn.commit()
    print("All tables created successfully.")

# Insert data into tables
def load_data_to_tables(datasets, dataset_names, conn):
    for i, dataset in enumerate(datasets):
        table_name = dataset_names[i].lower()
        print(f"Loading data into table: {table_name}")
        
        dataset.to_sql(table_name, conn, if_exists="replace", index=False)
        print(f"Data loaded into {table_name} successfully.")

# Verify the contents of the database
def verify_database(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

    # Example: Print the first few rows of each table
    for table in tables:
        print(f"\nData from table: {table[0]}")
        query = f"SELECT * FROM {table[0]} LIMIT 5;"
        result = pd.read_sql_query(query, conn)
        print(result)


# Create tables and load data
create_tables(datasets, dataset_names, key_definitions, conn)
load_data_to_tables(datasets, dataset_names, conn)

# Verify the contents of the database
verify_database(conn)

# Close connection
conn.close()


Creating table: customers

        CREATE TABLE IF NOT EXISTS customers (
            akeed_customer_id TEXT, gender TEXT, dob TEXT, status INTEGER, verified INTEGER, language TEXT, created_at DATETIME, updated_at DATETIME, PRIMARY KEY (akeed_customer_id)
        );
        
Creating table: orders

        CREATE TABLE IF NOT EXISTS orders (
            akeed_order_id REAL, customer_id TEXT, item_count REAL, grand_total REAL, payment_mode INTEGER, vendor_discount_amount REAL, is_favorite TEXT, is_rated TEXT, driver_rating REAL, deliverydistance REAL, preparationtime REAL, order_accepted_time DATETIME, ready_for_pickup_time DATETIME, picked_up_time DATETIME, delivered_time DATETIME, vendor_id INTEGER, created_at DATETIME, LOCATION_NUMBER INTEGER, LOCATION_TYPE TEXT, CID X LOC_NUM X VENDOR TEXT, PRIMARY KEY (akeed_order_id), FOREIGN KEY (customer_id) REFERENCES customers(akeed_customer_id), FOREIGN KEY (location_number, location_type) REFERENCES locations(location_number, location_type),